In [52]:
import os
import shutil
if os.path.isdir("3D files"):
    shutil.rmtree("3D files")
os.mkdir("3D files")

In [53]:
texture_wool = {
    "wool_colored_white": (201, 201, 201), 
    "wool_colored_silver":(121, 121, 121), 
    "wool_colored_gray":(62, 62, 62), 
    "wool_colored_black":(12, 12, 12), 
    "wool_colored_brown":(104, 66, 39), 
    "wool_colored_red":(118, 27, 26), 
    "wool_colored_orange":(183, 82, 5), 
    "wool_colored_yellow":(191, 140, 17),  
    "wool_colored_lime":(81, 140, 19), 
    "wool_colored_green":(61, 77, 26), 
    "wool_colored_cyan":(17, 104, 111), 
    "wool_colored_light_blue":(57, 153, 180), 
    "wool_colored_blue":(39, 40, 119), 
    "wool_colored_purple":(104, 37, 142),
    "wool_colored_magenta":(167, 76, 165), 
    "wool_colored_pink":(182, 95, 123)
}

texture_terracota = {
    "hardened_clay_stained_white":(164, 136, 126), 
    "hardened_clay_stained_silver":(107, 83, 76), 
    "hardened_clay_stained_gray":(47, 34, 28), 
    "hardened_clay_stained_black":(29, 18, 13), 
    "hardened_clay_stained_brown":(62, 42, 29), 
    "hardened_clay_stained_red":(115, 51, 38), 
    "hardened_clay_stained_orange":(130, 69, 32), 
    "hardened_clay_stained_yellow":(150, 108, 29),  
    "hardened_clay_stained_lime":(81, 92, 42), 
    "hardened_clay_stained_green":(60, 66, 33), 
    "hardened_clay_stained_cyan":(67, 70, 71), 
    "hardened_clay_stained_light_blue":(90, 85, 109), 
    "hardened_clay_stained_blue":(58, 47, 72), 
    "hardened_clay_stained_purple":(93, 55, 68), 
    "hardened_clay_stained_magenta":(122, 72, 87), 
    "hardened_clay_stained_pink":(128, 62, 62)
}

texture_others = {
    "prismarine_bricks":(87, 141, 130), 
    "emerald_block":(79, 185, 109), 
    "sponge":(159, 157, 75), 
    "stone":(100, 100, 100), 
    "sandstone_normal":(172, 166, 123), 
    "dirt":(96, 67, 56),
    "sponge_wet":(118, 115, 37), 
    "purpur_block":(131, 93, 131), 
    "stone_granite_smooth":(130, 91, 76),
    "diamond_block":(107,181,178),
}


all_textures = [texture_wool, texture_terracota, texture_others]

In [54]:
import math

def closest_pixel_distance(pixel_image):
    min_dist = 10000
    best_texture = ""
    pixel_value = None
    for texture_pack in all_textures:
        for texture, pixel in texture_pack.items():
            dist = math.sqrt((pixel[0] - pixel_image[0])**2 + (pixel[1] - pixel_image[1])**2 + (pixel[2] - pixel_image[2])**2)
            if dist < min_dist:
                min_dist = dist
                best_texture = texture
                pixel_value = pixel
    return best_texture, pixel_value

In [55]:
from PIL import Image

im = Image.open('Coucou_cropped.png')
im.thumbnail((im.size[0] / 8, im.size[1] / 8))
pixels = list(im.getdata())
width, height = im.size
pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]

In [56]:
new_pixels = []
textures = []
for line in pixels:
    l = [closest_pixel_distance(pixel)[1] for pixel in line]
    l2 = [closest_pixel_distance(pixel)[0] for pixel in line]
    new_pixels.append(l)
    textures.append(l2)

In [57]:
import numpy as np

array = np.array(new_pixels, dtype=np.uint8)
new_image = Image.fromarray(array)
new_image.save('new.png')

In [58]:
width, height = len(new_pixels[0]), len(new_pixels)

textures_dict = {}
pixels_dict = {}

block_size = 4
size = 16 // block_size

for i, line in enumerate(new_pixels):
    if i // block_size not in pixels_dict:
        pixels_dict[i // block_size] = {}
    for j, pixel in enumerate(line):
        if j // block_size not in pixels_dict[i // block_size]:
            pixels_dict[i // block_size][j // block_size] = {}
        if i % block_size not in pixels_dict[i // block_size][j // block_size]:
            pixels_dict[i // block_size][j // block_size][i % block_size] = []
        pixels_dict[i // block_size][j // block_size][i % block_size].append(pixel)


for i, line in enumerate(textures):
    if i // block_size not in textures_dict:
        textures_dict[i // block_size] = {}
    for j, texture in enumerate(line):
        if j // block_size not in textures_dict[i // block_size]:
            textures_dict[i // block_size][j // block_size] = {}
        if i % block_size not in textures_dict[i // block_size][j // block_size]:
            textures_dict[i // block_size][j // block_size][i % block_size] = []
        textures_dict[i // block_size][j // block_size][i % block_size].append(texture)


In [59]:
len(textures_dict), len(textures_dict[0]), len(textures_dict[0][0])

(21, 43, 4)

In [60]:
def find_background(block):
    list_textures = {}
    for i, texture_list in block.items():
        for texture in texture_list:
            if texture not in list_textures:
                list_textures[texture] = 1
            else:
                list_textures[texture] += 1
    max = 0
    background = ""
    for texture, i in list_textures.items():
        if i > max:
            max = i
            background = texture
    return background

In [61]:
find_background(textures_dict[0][0])

'sponge'

In [62]:
def indices_in(corners, l):
    for m in l:
        if (m[0][0] <= corners[0][0] and m[0][1] <= corners[0][1]) or (m[1][0] >= corners[1][0] and m[1][1] >= corners[1][1]):
            return True
    return False

In [63]:
def find_bigger_blocks(block):
    sub_blocks = []
    textures_matrix = np.array([[pixel for pixel in list_pixel] for i, list_pixel in block.items()])
    for i in range(5, 1, -1):
        for j in range(5, 1, -1):
            for i2 in range(len(textures_matrix) - i + 1):
                for j2 in range(len(textures_matrix[0]) - j + 1):
                    test_matrix = textures_matrix[i2:i2+i, j2:j2+j]
                    if test_matrix[0][0] != None and np.all(test_matrix == test_matrix[0][0]):
                        if not indices_in(((i2, j2), (i2 + i - 1, j2 + j - 1)), sub_blocks):
                            sub_blocks.append(((i2, j2), (i2+i-1, j2+j-1)))
    return sub_blocks
        

In [64]:
find_bigger_blocks(textures_dict[0][0])

[((0, 0), (1, 2))]

In [65]:
for i, texture_line in enumerate(textures_dict):
    for j, texture_block in enumerate(textures_dict[texture_line]):
        lenght_shapes = 1
        textures = textures_dict[texture_line][texture_block]
        background = find_background(textures)
        long_str = "{\n\t" + f'label="Coucou",\n\ttooltip="block-{i},{j}",\n\tlightlevel=8,\n\tshapes=' + '{\n'
        long_str += "\t\t{" + f'0, 0, 15, 16, 16, 16, texture="{background}"' + "},\n"
        for i2, texture_list in textures.items():
            for j2, texture in enumerate(texture_list):
                if texture == background:
                    texture_list[j2] = None
        bigger_blocks = find_bigger_blocks(textures)
        for block in bigger_blocks:
            texture = textures[block[0][0]][block[0][1]]
            long_str += "\t\t{" + f'{size*block[0][0]}, {16-size*block[0][1]}, 14, {size*block[1][0]+size}, {16-size*block[1][1]-size}, 15, texture="{texture}"' + "},\n"
        for i2, texture_list in textures.items():
            for j2, texture in enumerate(texture_list):
                if indices_in(((i2,j2),(i2,j2)), bigger_blocks):
                    texture_list[j2] = None
        lenght_shapes += len(bigger_blocks)
        
        for k, texture_list in textures.items():
            for l, texture in enumerate(texture_list):
                if texture is not None:
                    long_str += "\t\t{" + f'{size*l}, {16-size*k}, 14, {size*l+size}, {16-size*k-size}, 15, texture="{texture}"' + "},\n"
                    lenght_shapes += 1
        long_str += "\t}\n}"
        if lenght_shapes > 20:
            print(lenght_shapes)
            print(long_str)
            break
        with open(f"3D files\\block_{i}-{j}.3dm", "w+") as f:
            f.write(long_str)
    if lenght_shapes > 20:
        break